In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import selenium

In [4]:
df = pd.read_parquet("../data/reservas_hoteles.parquet")
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [6]:
df.isnull().sum()

id_reserva            0
id_cliente            0
nombre                0
apellido              0
mail                  0
competencia           0
fecha_reserva         0
inicio_estancia      75
final_estancia       75
id_hotel              0
precio_noche       5224
nombre_hotel          0
estrellas          5172
ciudad                0
dtype: int64

In [7]:
df["ciudad"] = "Madrid"
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [8]:
df["inicio_estancia"] = df["inicio_estancia"][0]
df["final_estancia"] = df["final_estancia"][0]

In [9]:
df.isnull().sum()

id_reserva            0
id_cliente            0
nombre                0
apellido              0
mail                  0
competencia           0
fecha_reserva         0
inicio_estancia       0
final_estancia        0
id_hotel              0
precio_noche       5224
nombre_hotel          0
estrellas          5172
ciudad                0
dtype: int64

In [10]:
url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [11]:
def scrap_hoteles(url):
    """
    Función para extraer información sobre hoteles desde una URL utilizando web scraping con BeautifulSoup.

    Esta función realiza una solicitud HTTP a la URL proporcionada, obtiene el contenido HTML de la página, y extrae
    los nombres de los hoteles y sus estrellas desde los elementos `<h2>` que contienen esta información.
    La función asume que el texto en cada `<h2>` está separado por un salto de línea (`\n`), donde la primera parte es 
    el nombre del hotel y la segunda parte es la cantidad de estrellas del hotel.

    Parámetros:
    ----------
    url : str
        La URL de la página web desde la cual se desea hacer scraping.

    Retorna:
    --------
    dict_hoteles : dict
        Un diccionario que contiene dos claves:
        - 'nombres_hotel': una lista con los nombres de los hoteles extraídos de la página.
        - 'estrellas_hotel': una lista con la cantidad de estrellas de cada hotel.
    
    Si la solicitud HTTP falla (código de estado diferente a 200), la función retorna un mensaje de error.

    Ejemplo:
    --------
    url = "https://ejemplo.com/hoteles"
    resultado = scrap_hoteles(url)
    
    print(resultado)
    # {'nombres_hotel': ['Mercure Madrid Centro', 'Hotel XYZ'], 'estrellas_hotel': ['4 Estrellas', '5 Estrellas']}
    """
    dict_hoteles = {
        "nombre_hotel": [],
        "estrellas": []
    }
    
    # Realizar solicitud HTTP
    res = requests.get(url)
    
    # Verificar que la solicitud fue exitosa (código 200)
    if res.status_code == 200:
        sopa_hoteles = BeautifulSoup(res.text, "html.parser")
        
        # Buscar todos los elementos <h2> que contienen la información de los hoteles
        headings = sopa_hoteles.find_all("h2")
        
        for head in headings:
            # Obtener el texto y eliminar espacios al principio y final
            text = head.get_text().strip()
            parts = text.split("\n")
            
            # Manejar el caso donde hay menos de dos partes
            if len(parts) >= 2:
                nombres_hotel = parts[0].strip()  # El nombre del hotel es la primera parte
                estrellas_hotel = parts[1].strip()  # La cantidad de estrellas es la segunda parte
            else:
                nombres_hotel = parts[0].strip()  # Si solo hay el nombre del hotel
                estrellas_hotel = "No rating available"  # Asignar valor por defecto para la estrella si falta
            
            # Añadir los datos al diccionario
            dict_hoteles["nombre_hotel"].append(nombres_hotel)
            dict_hoteles["estrellas"].append(estrellas_hotel)
    
    else:
        return f"No se ha podido encontrar la url, status code: {res.status_code}"
    
    return dict_hoteles
    

In [12]:
dicti_hoteles = pd.DataFrame(scrap_hoteles(url))
dicti_hoteles

,nombre_hotel,estrellas
0,ibis Styles Madrid Prado,3 Estrellas
1,Mercure Madrid Centro,4 Estrellas
2,ibis budget Madrid Centro Lavapies,1 Estrellas
3,ibis Styles Madrid Centro Maravillas,2 Estrellas
4,Novotel Madrid Center,4 Estrellas
5,ibis budget Madrid Calle 30,1 Estrellas
6,ibis Madrid Centro las Ventas,2 Estrellas
7,ibis budget Madrid Centro las Ventas,1 Estrellas
8,Novotel Madrid City Las Ventas,4 Estrellas
9,ibis Styles Madrid City Las Ventas,4 Estrellas


In [13]:
id_usados = range(0, 20)
id_disponibles = [id for id in df["id_hotel"] if id not in id_usados]

In [14]:
id_disponibles_unicos = []
for i in id_disponibles:
    if i not in id_disponibles_unicos:
        id_disponibles_unicos.append(i)


In [15]:
print(id_disponibles_unicos)

[113, 194, 38, 35, 36, 43, 131, 114, 103, 181, 29, 47, 128, 27, 28, 40, 186, 48, 135, 33, 117, 49]


In [16]:
dicti_hoteles["id_hotel"] = id_disponibles_unicos[:len(dicti_hoteles)]


In [17]:
dicti_hoteles

,nombre_hotel,estrellas,id_hotel
0,ibis Styles Madrid Prado,3 Estrellas,113
1,Mercure Madrid Centro,4 Estrellas,194
2,ibis budget Madrid Centro Lavapies,1 Estrellas,38
3,ibis Styles Madrid Centro Maravillas,2 Estrellas,35
4,Novotel Madrid Center,4 Estrellas,36
5,ibis budget Madrid Calle 30,1 Estrellas,43
6,ibis Madrid Centro las Ventas,2 Estrellas,131
7,ibis budget Madrid Centro las Ventas,1 Estrellas,114
8,Novotel Madrid City Las Ventas,4 Estrellas,103
9,ibis Styles Madrid City Las Ventas,4 Estrellas,181


In [18]:
df_merge = pd.merge(df, dicti_hoteles, on="id_hotel", how="left", suffixes=("", "_corrected"))
df_merge["nombre_hotel"] = df_merge["nombre_hotel"].fillna(df_merge["nombre_hotel_corrected"])
df_merge["estrellas"] = df_merge["estrellas"].fillna(df_merge["estrellas_corrected"])
df_merge.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_hotel_corrected,estrellas_corrected
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,3 Estrellas,Madrid,ibis Styles Madrid Prado,3 Estrellas
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,4 Estrellas,Madrid,Mercure Madrid Centro,4 Estrellas
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,NaN,NaN
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,ibis budget Madrid Centro Lavapies,1 Estrellas
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,ibis Styles Madrid Centro Maravillas,2 Estrellas


In [19]:
df_merge = df_merge.drop(columns= ["nombre_hotel_corrected", "estrellas_corrected"])
df_merge.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,3 Estrellas,Madrid
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,4 Estrellas,Madrid
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [20]:
df_merge["fecha_reserva"].fillna(df_merge["fecha_reserva"].iloc[2], inplace=True)

/var/folders/r4/jsr6jl3d5js8g1ztlswntdr80000gp/T/ipykernel_35624/1440355124.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge["fecha_reserva"].fillna(df_merge["fecha_reserva"].iloc[2], inplace=True)


In [21]:
df_merge.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,3 Estrellas,Madrid
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,4 Estrellas,Madrid
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [22]:
df_merge["fecha_reserva"].replace("", df_merge["fecha_reserva"].iloc[2], inplace=True)

/var/folders/r4/jsr6jl3d5js8g1ztlswntdr80000gp/T/ipykernel_35624/1400894565.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge["fecha_reserva"].replace("", df_merge["fecha_reserva"].iloc[2], inplace=True)


In [23]:
df_merge.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-09,2025-03-01,2025-03-02,113,NaN,,3 Estrellas,Madrid
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-09,2025-03-01,2025-03-02,194,NaN,,4 Estrellas,Madrid
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
